In [1]:
!pip install DI-engine

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.3/185.3 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 732.2/732.2 kB 11.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 24.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/10

In [6]:
!apt-get install swig3.0
!ln -s /usr/bin/swig3.0 /usr/bin/swig

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
swig3.0 is already the newest version (3.0.12-2.2ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [7]:
!pip install gym
!pip install gym[box2d]

  Using cached box2d-py-2.3.5.tar.gz (374 kB)
  Preparing metadata (setup.py) ... done
  Using cached pygame-2.1.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Created wheel for box2d-py: filename=box2d_py-2.3.5-cp310-cp310-linux_x86_64.whl size=2309694 sha256=6558261a35a96998135d212fc7502b73722c42b25e160805885fd5273536aa43
  Stored in directory: /root/.cache/pip/wheels/db/8f/6a/eaaadf056fba10a98d986f6dce954e6201ba3126926fc5ad9e
Successfully built box2d-py
  Attempting uninstall: pygame
    Found existing installation: pygame 2.5.2
    Uninstalling pygame-2.5.2:
      Successfully uninstalled pygame-2.5.2


# Cartpole (for reference)

In [23]:
from dizoo.classic_control.cartpole.config.cartpole_dqn_config import main_config, create_config
from ding.config import compile_config
import gym
cfg = compile_config(main_config, create_cfg=create_config, auto=True)

In [24]:
from ding.envs import DingEnvWrapper, BaseEnvManagerV2

collector_env = BaseEnvManagerV2(
    env_fn=[lambda: DingEnvWrapper(gym.make("CartPole-v0")) for _ in range(cfg.env.collector_env_num)],
    cfg=cfg.env.manager
)
evaluator_env = BaseEnvManagerV2(
    env_fn=[lambda: DingEnvWrapper(gym.make("CartPole-v0")) for _ in range(cfg.env.evaluator_env_num)],
    cfg=cfg.env.manager
)

/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(


In [25]:
from ding.model import DQN
from ding.policy import DQNPolicy
from ding.data import DequeBuffer

model = DQN(**cfg.policy.model)
buffer_ = DequeBuffer(size=cfg.policy.other.replay_buffer.replay_buffer_size)
policy = DQNPolicy(cfg.policy, model=model)

In [26]:
from ding.framework import task
from ding.framework.context import OnlineRLContext
from ding.framework.middleware import OffPolicyLearner, StepCollector, interaction_evaluator, data_pusher, eps_greedy_handler, CkptSaver

import logging
logging.getLogger().setLevel(logging.INFO)

with task.start(async_mode=False, ctx=OnlineRLContext()):
    # Evaluating, we place it on the first place to get the score of the random model as a benchmark value
    task.use(interaction_evaluator(cfg, policy.eval_mode, evaluator_env))
    task.use(eps_greedy_handler(cfg))  # Decay probability of explore-exploit
    task.use(StepCollector(cfg, policy.collect_mode, collector_env))  # Collect environmental data
    task.use(data_pusher(cfg, buffer_))  # Push data to buffer
    task.use(OffPolicyLearner(cfg, policy.learn_mode, buffer_))  # Train the model
    task.use(CkptSaver(policy, cfg.exp_name, train_freq=100))  # Save the model
    # In the evaluation process, if the model is found to have exceeded the convergence value, it will end early here
    task.run()

INFO:root:Env Space Information:
INFO:root:	Observation Space: Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)
INFO:root:	Action Space: Discrete(2)
INFO:root:	Reward Space: Box(-inf, inf, (1,), float32)
/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step

# serial pipeline and GAIL prep

In [27]:
# gail serial pipeline
from typing import Optional, Tuple
import os
import torch
from ditk import logging
from functools import partial
from tensorboardX import SummaryWriter
from copy import deepcopy
import numpy as np

from ding.envs import get_vec_env_setting, create_env_manager
from ding.worker import BaseLearner, InteractionSerialEvaluator, BaseSerialCommander, create_buffer, \
    create_serial_collector
from ding.config import read_config, compile_config
from ding.policy import create_policy
from ding.reward_model import create_reward_model
from ding.utils import set_pkg_seed
from ding.entry import collect_demo_data
from ding.utils import save_file
#from .utils import random_collect


def save_reward_model(path, reward_model, weights_name='best'):
    path = os.path.join(path, 'reward_model', 'ckpt')
    if not os.path.exists(path):
        try:
            os.makedirs(path)
        except FileExistsError:
            pass
    path = os.path.join(path, 'ckpt_{}.pth.tar'.format(weights_name))
    state_dict = reward_model.state_dict()
    save_file(path, state_dict)
    print('Saved reward model ckpt in {}'.format(path))


def serial_pipeline_gail(
        input_cfg: Tuple[dict, dict],
        expert_cfg: Tuple[dict, dict],
        seed: int = 0,
        model: Optional[torch.nn.Module] = None,
        max_train_iter: Optional[int] = int(1e10),
        max_env_step: Optional[int] = int(1e10),
        collect_data: bool = True,
) -> 'Policy':  # noqa
    """
    Overview:
        Serial pipeline entry for GAIL reward model.
    Arguments:
        - input_cfg (:obj:`Union[str, Tuple[dict, dict]]`): Config in dict type. \
            ``str`` type means config file path. \
            ``Tuple[dict, dict]`` type means [user_config, create_cfg].
        - expert_cfg (:obj:`Union[str, Tuple[dict, dict]]`): Expert config in dict type. \
            ``str`` type means config file path. \
            ``Tuple[dict, dict]`` type means [user_config, create_cfg].
        - seed (:obj:`int`): Random seed.
        - model (:obj:`Optional[torch.nn.Module]`): Instance of torch.nn.Module.
        - max_train_iter (:obj:`Optional[int]`): Maximum policy update iterations in training.
        - max_env_step (:obj:`Optional[int]`): Maximum collected environment interaction steps.
        - collect_data (:obj:`bool`): Collect expert data.
    Returns:
        - policy (:obj:`Policy`): Converged policy.
    """
    if isinstance(input_cfg, str):
        cfg, create_cfg = read_config(input_cfg)
    else:
        cfg, create_cfg = deepcopy(input_cfg)
    if isinstance(expert_cfg, str):
        expert_cfg, expert_create_cfg = read_config(expert_cfg)
    else:
        expert_cfg, expert_create_cfg = expert_cfg
    create_cfg.policy.type = create_cfg.policy.type + '_command'
    cfg = compile_config(cfg, seed=seed, auto=True, create_cfg=create_cfg, save_cfg=True)
    if 'data_path' not in cfg.reward_model:
        cfg.reward_model.data_path = cfg.exp_name
    # Load expert data
    if collect_data:
        if expert_cfg.policy.get('other', None) is not None and expert_cfg.policy.other.get('eps', None) is not None:
            expert_cfg.policy.other.eps.collect = -1
        if expert_cfg.policy.get('load_path', None) is None:
            expert_cfg.policy.load_path = cfg.reward_model.expert_model_path
        collect_demo_data(
            (expert_cfg, expert_create_cfg),
            seed,
            state_dict_path=expert_cfg.policy.load_path,
            expert_data_path=cfg.reward_model.data_path + '/expert_data.pkl',
            collect_count=cfg.reward_model.collect_count
        )
    # Create main components: env, policy
    env_fn, collector_env_cfg, evaluator_env_cfg = get_vec_env_setting(cfg.env)
    collector_env = create_env_manager(cfg.env.manager, [partial(env_fn, cfg=c) for c in collector_env_cfg])
    evaluator_env = create_env_manager(cfg.env.manager, [partial(env_fn, cfg=c) for c in evaluator_env_cfg])
    collector_env.seed(cfg.seed)
    evaluator_env.seed(cfg.seed, dynamic_seed=False)
    set_pkg_seed(cfg.seed, use_cuda=cfg.policy.cuda)
    policy = create_policy(cfg.policy, model=model, enable_field=['learn', 'collect', 'eval', 'command'])

    # Create worker components: learner, collector, evaluator, replay buffer, commander.
    tb_logger = SummaryWriter(os.path.join('./{}/log/'.format(cfg.exp_name), 'serial'))
    learner = BaseLearner(cfg.policy.learn.learner, policy.learn_mode, tb_logger, exp_name=cfg.exp_name)
    collector = create_serial_collector(
        cfg.policy.collect.collector,
        env=collector_env,
        policy=policy.collect_mode,
        tb_logger=tb_logger,
        exp_name=cfg.exp_name
    )
    evaluator = InteractionSerialEvaluator(
        cfg.policy.eval.evaluator, evaluator_env, policy.eval_mode, tb_logger, exp_name=cfg.exp_name
    )
    replay_buffer = create_buffer(cfg.policy.other.replay_buffer, tb_logger=tb_logger, exp_name=cfg.exp_name)
    commander = BaseSerialCommander(
        cfg.policy.other.commander, learner, collector, evaluator, replay_buffer, policy.command_mode
    )
    reward_model = create_reward_model(cfg.reward_model, policy.collect_mode.get_attribute('device'), tb_logger)

    # ==========
    # Main loop
    # ==========
    # Learner's before_run hook.
    learner.call_hook('before_run')

    # Accumulate plenty of data at the beginning of training.
    if cfg.policy.get('random_collect_size', 0) > 0:
        random_collect(cfg.policy, policy, collector, collector_env, commander, replay_buffer)
    best_reward = -np.inf
    while True:
        collect_kwargs = commander.step()
        # Evaluate policy performance
        if evaluator.should_eval(learner.train_iter):
            stop, reward = evaluator.eval(learner.save_checkpoint, learner.train_iter, collector.envstep)
            print("look here")
            print(reward)
            # reward_mean = np.array([r['eval_episode_return'] for r in reward]).mean()
            reward_mean = np.array(reward['eval_episode_return']).mean()
            if reward_mean >= best_reward:
                save_reward_model(cfg.exp_name, reward_model, 'best')
                best_reward = reward_mean
            if stop:
                break
        new_data_count, target_new_data_count = 0, cfg.reward_model.get('target_new_data_count', 1)
        while new_data_count < target_new_data_count:
            new_data = collector.collect(train_iter=learner.train_iter, policy_kwargs=collect_kwargs)
            new_data_count += len(new_data)
            # collect data for reward_model training
            reward_model.collect_data(new_data)
            replay_buffer.push(new_data, cur_collector_envstep=collector.envstep)
        # update reward_model
        reward_model.train()
        reward_model.clear_data()
        # Learn policy from collected data
        for i in range(cfg.policy.learn.update_per_collect):
            # Learner will train ``update_per_collect`` times in one iteration.
            train_data = replay_buffer.sample(learner.policy.get_attribute('batch_size'), learner.train_iter)
            if train_data is None:
                # It is possible that replay buffer's data count is too few to train ``update_per_collect`` times
                logging.warning(
                    "Replay buffer's data can only train for {} steps. ".format(i) +
                    "You can modify data collect config, e.g. increasing n_sample, n_episode."
                )
                break
            # update train_data reward using the augmented reward
            train_data_augmented = reward_model.estimate(train_data)
            learner.train(train_data_augmented, collector.envstep)
            if learner.policy.get_attribute('priority'):
                replay_buffer.update(learner.priority_info)
        if collector.envstep >= max_env_step or learner.train_iter >= max_train_iter:
            break

    # Learner's after_run hook.
    learner.call_hook('after_run')
    save_reward_model(cfg.exp_name, reward_model, 'last')
    # evaluate
    # evaluator.eval(learner.save_checkpoint, learner.train_iter, collector.envstep)
    return policy

In [28]:
from typing import Optional, Callable, List, Any

from ding.policy import PolicyFactory
from ding.worker import IMetric, MetricSerialEvaluator


class AccMetric(IMetric):

    def eval(self, inputs: Any, label: Any) -> dict:
        return {'Acc': (inputs['logit'].sum(dim=1) == label).sum().item() / label.shape[0]}

    def reduce_mean(self, inputs: List[Any]) -> Any:
        s = 0
        for item in inputs:
            s += item['Acc']
        return {'Acc': s / len(inputs)}

    def gt(self, metric1: Any, metric2: Any) -> bool:
        if metric2 is None:
            return True
        if isinstance(metric2, dict):
            m2 = metric2['Acc']
        else:
            m2 = metric2
        return metric1['Acc'] > m2


def mark_not_expert(ori_data: List[dict]) -> List[dict]:
    for i in range(len(ori_data)):
        # Set is_expert flag (expert 1, agent 0)
        ori_data[i]['is_expert'] = 0
    return ori_data


def mark_warm_up(ori_data: List[dict]) -> List[dict]:
    # for td3_vae
    for i in range(len(ori_data)):
        ori_data[i]['warm_up'] = True
    return ori_data


def random_collect( #粘过来是为了用这个
        policy_cfg: 'EasyDict',  # noqa
        policy: 'Policy',  # noqa
        collector: 'ISerialCollector',  # noqa
        collector_env: 'BaseEnvManager',  # noqa
        commander: 'BaseSerialCommander',  # noqa
        replay_buffer: 'IBuffer',  # noqa
        postprocess_data_fn: Optional[Callable] = None
) -> None:  # noqa
    assert policy_cfg.random_collect_size > 0
    if policy_cfg.get('transition_with_policy_data', False):
        collector.reset_policy(policy.collect_mode)
    else:
        action_space = collector_env.action_space
        random_policy = PolicyFactory.get_random_policy(policy.collect_mode, action_space=action_space)
        collector.reset_policy(random_policy)
    collect_kwargs = commander.step()
    if policy_cfg.collect.collector.type == 'episode':
        new_data = collector.collect(n_episode=policy_cfg.random_collect_size, policy_kwargs=collect_kwargs)
    else:
        new_data = collector.collect(
            n_sample=policy_cfg.random_collect_size,
            random_collect=True,
            record_random_collect=False,
            policy_kwargs=collect_kwargs
        )  # 'record_random_collect=False' means random collect without output log
    if postprocess_data_fn is not None:
        new_data = postprocess_data_fn(new_data)
    replay_buffer.push(new_data, cur_collector_envstep=0)
    collector.reset_policy(policy.collect_mode)

# GAIL SAC bipedalwalker

In [43]:
# dizoo/box2d/bipedalwalker/config/bipedalwalker_gail_sac_config.py
from easydict import EasyDict

obs_shape = 24
act_shape = 4
bipedalwalker_sac_gail_default_config = dict(
    exp_name='bipedalwalker_sac_gail_seed0',
    env=dict(
        collector_env_num=8,
        evaluator_env_num=5,
        # (bool) Scale output action into legal range.
        act_scale=True,
        n_evaluator_episode=5,
        stop_value=300,
        rew_clip=True,
        # The path to save the game replay
        replay_path=None,
    ),
    reward_model=dict(
        type='gail',
        input_size=obs_shape + act_shape,
        hidden_size=64,
        batch_size=64,
        learning_rate=1e-3,
        update_per_collect=100,
        # Users should add their own model path here. Model path should lead to a model.
        # Absolute path is recommended.
        # In DI-engine, it is ``exp_name/ckpt/ckpt_best.pth.tar``.
        # expert_model_path='model_path_placeholder',
        expert_model_path='bipedalwalker_sac_config0/ckpt/ckpt_best.pth.tar',
        # Path where to store the reward model
        reward_model_path='bipedalwalker_sac_gail_seed0/reward_model/ckpt/ckpt_best.pth.tar',
        # Users should add their own data path here. Data path should lead to a file to store data or load the stored data.
        # Absolute path is recommended.
        # In DI-engine, it is usually located in ``exp_name`` directory
        data_path='bipedalwalker_sac_gail_seed0',
        collect_count=100000,
    ),
    policy=dict(
        cuda=False,
        priority=False,
        random_collect_size=1000,
        model=dict(
            obs_shape=obs_shape,
            action_shape=act_shape,
            twin_critic=True,
            action_space='reparameterization',
            actor_head_hidden_size=128,
            critic_head_hidden_size=128,
        ),
        learn=dict(
            update_per_collect=1,
            batch_size=128,
            learning_rate_q=0.001,
            learning_rate_policy=0.001,
            learning_rate_alpha=0.0003,
            ignore_done=True,
            target_theta=0.005,
            discount_factor=0.99,
            auto_alpha=True,
            value_network=False,
        ),
        collect=dict(
            n_sample=128,
            unroll_len=1,
        ),
        other=dict(replay_buffer=dict(replay_buffer_size=100000, ), ),
    ),
)
bipedalwalker_sac_gail_default_config = EasyDict(bipedalwalker_sac_gail_default_config)
main_config = bipedalwalker_sac_gail_default_config

bipedalwalker_sac_gail_create_config = dict(
    env=dict(
        type='bipedalwalker',
        import_names=['dizoo.box2d.bipedalwalker.envs.bipedalwalker_env'],
    ),
    env_manager=dict(type='subprocess'),
    policy=dict(
        type='sac',
        import_names=['ding.policy.sac'],
    ),
    replay_buffer=dict(type='naive', ),
)
bipedalwalker_sac_gail_create_config = EasyDict(bipedalwalker_sac_gail_create_config)
create_config = bipedalwalker_sac_gail_create_config


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [44]:
# dizoo.box2d.bipedalwalker.config bipedalwalker_sac_config
from easydict import EasyDict

bipedalwalker_sac_config = dict(
    exp_name='bipedalwalker_sac_config0',
    env=dict(
        env_id='BipedalWalker-v3',
        collector_env_num=8,
        evaluator_env_num=5,
        # (bool) Scale output action into legal range.
        act_scale=True,
        n_evaluator_episode=5,
        rew_clip=True,
    ),
    policy=dict(
        cuda=True,
        random_collect_size=10000,
        model=dict(
            obs_shape=24,
            action_shape=4,
            twin_critic=True,
            action_space='reparameterization',
            actor_head_hidden_size=128,
            critic_head_hidden_size=128,
        ),
        learn=dict(
            update_per_collect=64,
            batch_size=256,
            learning_rate_q=0.0003,
            learning_rate_policy=0.0003,
            learning_rate_alpha=0.0003,
            target_theta=0.005,
            discount_factor=0.99,
            auto_alpha=True,
            learner=dict(hook=dict(log_show_after_iter=1000, ))
        ),
        collect=dict(n_sample=64, ),
        other=dict(replay_buffer=dict(replay_buffer_size=300000, ), ),
    ),
)
bipedalwalker_sac_config = EasyDict(bipedalwalker_sac_config)
expert_main_config = bipedalwalker_sac_config
bipedalwalker_sac_create_config = dict(
    env=dict(
        type='bipedalwalker',
        import_names=['dizoo.box2d.bipedalwalker.envs.bipedalwalker_env'],
    ),
    env_manager=dict(type='subprocess'),
    policy=dict(type='sac', ),
    replay_buffer=dict(type='naive', ),
)
bipedalwalker_sac_create_config = EasyDict(bipedalwalker_sac_create_config)
expert_create_config = bipedalwalker_sac_create_config

In [76]:
#from ding.entry import serial_pipeline_gail
#from dizoo.box2d.bipedalwalker.config import bipedalwalker_sac_config , bipedalwalker_sac_create_config
#expert_main_config = bipedalwalker_sac_config
#expert_create_config = bipedalwalker_sac_create_config
serial_pipeline_gail(
    [main_config, create_config],
    [expert_main_config, expert_create_config],
    seed=0,
    collect_data=True
)

Exception ignored in: <function SampleSerialCollector.__del__ at 0x784d49501f30>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/ding/worker/collector/sample_serial_collector.py", line 212, in __del__
    self.close()
  File "/usr/local/lib/python3.10/dist-packages/ding/worker/collector/sample_serial_collector.py", line 201, in close
    self._env.close()
  File "/usr/local/lib/python3.10/dist-packages/ding/envs/env_manager/subprocess_env_manager.py", line 635, in close
    p.send(['close', None, None])
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 206, in send
    self._send_bytes(_ForkingPickler.dumps(obj))
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 411, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Exception ignored in: <function InteractionSerialEvaluator.__de

Collect demo data successfully


INFO:learner_logger:[RANK0]: DI-engine DRL Policy
ContinuousQAC(
  (actor_encoder): Identity()
  (critic_encoder): Identity()
  (actor_head): Sequential(
    (0): Linear(in_features=24, out_features=128, bias=True)
    (1): ReLU()
    (2): ReparameterizationHead(
      (main): Sequential(
        (0): Linear(in_features=128, out_features=128, bias=True)
        (1): ReLU()
      )
      (mu): Linear(in_features=128, out_features=4, bias=True)
      (log_sigma_layer): Linear(in_features=128, out_features=4, bias=True)
    )
  )
  (critic_head): ModuleList(
    (0-1): 2 x Sequential(
      (0): Linear(in_features=28, out_features=128, bias=True)
      (1): ReLU()
      (2): RegressionHead(
        (main): Sequential(
          (0): Linear(in_features=128, out_features=128, bias=True)
          (1): ReLU()
        )
        (last): Linear(in_features=128, out_features=1, bias=True)
      )
    )
  )
  (actor): ModuleList(
    (0): Identity()
    (1): Sequential(
      (0): Linear(in_featu

look here
{'eval_episode_return': [-91.96858215332031, -91.98682403564453, -91.91168212890625, -91.9603500366211, -92.05400085449219], 'eval_episode_return_mean': -91.97628784179688}
Saved reward model ckpt in bipedalwalker_sac_gail_seed0_240229_005622/reward_model/ckpt/ckpt_best.pth.tar


INFO:learner_logger:[RANK0]: === Training Iteration 0 Result ===
INFO:learner_logger:
+-------+--------------------------+-----------------+-----------------+--------------+
| Name  | value_lossalpha_loss_avg | policy_loss_avg | critic_loss_avg | cur_lr_q_avg |
+-------+--------------------------+-----------------+-----------------+--------------+
| Value | 0.000000                 | -0.648018       | 0.922292        | 0.001000     |
+-------+--------------------------+-----------------+-----------------+--------------+
+-------+--------------+--------------------+-----------+--------------+
| Name  | cur_lr_p_avg | target_q_value_avg | alpha_avg | td_error_avg |
+-------+--------------+--------------------+-----------+--------------+
| Value | 0.001000     | 0.580234           | 0.199940  | 0.973497     |
+-------+--------------+--------------------+-----------+--------------+
+-------+--------------------------+----------------------+----------------+
| Name  | transformed_log_prob_a

...

INFO:learner_logger:[RANK0]: === Training Iteration 8000 Result ===
INFO:learner_logger:
+-------+--------------------------+-----------------+-----------------+--------------+
| Name  | value_lossalpha_loss_avg | policy_loss_avg | critic_loss_avg | cur_lr_q_avg |
+-------+--------------------------+-----------------+-----------------+--------------+
| Value | 0.000000                 | -77.495364      | 22.831909       | 0.001000     |
+-------+--------------------------+-----------------+-----------------+--------------+
+-------+--------------+--------------------+-----------+--------------+
| Name  | cur_lr_p_avg | target_q_value_avg | alpha_avg | td_error_avg |
+-------+--------------+--------------------+-----------+--------------+
| Value | 0.001000     | 76.512214          | 0.058165  | 22.970328    |
+-------+--------------+--------------------+-----------+--------------+
+-------+--------------------------+----------------------+----------------+
| Name  | transformed_log_pro

look here
{'eval_episode_return': [-81.85468292236328, -84.48780822753906, -84.23902893066406, -78.72492980957031, -82.0257568359375], 'eval_episode_return_mean': -82.26644134521484}


INFO:learner_logger:[RANK0]: === Training Iteration 9000 Result ===
INFO:learner_logger:
+-------+--------------------------+-----------------+-----------------+--------------+
| Name  | value_lossalpha_loss_avg | policy_loss_avg | critic_loss_avg | cur_lr_q_avg |
+-------+--------------------------+-----------------+-----------------+--------------+
| Value | 0.000000                 | -99.850839      | 26.323276       | 0.001000     |
+-------+--------------------------+-----------------+-----------------+--------------+
+-------+--------------+--------------------+-----------+--------------+
| Name  | cur_lr_p_avg | target_q_value_avg | alpha_avg | td_error_avg |
+-------+--------------+--------------------+-----------+--------------+
| Value | 0.001000     | 92.108272          | 0.064726  | 26.003213    |
+-------+--------------+--------------------+-----------+--------------+
+-------+--------------------------+----------------------+----------------+
| Name  | transformed_log_pro

look here
{'eval_episode_return': [305.284912109375, 304.89337158203125, 306.40399169921875, 305.0522155761719, 303.91790771484375], 'eval_episode_return_mean': 305.1104797363281}
Saved reward model ckpt in bipedalwalker_sac_gail_seed0_240229_005622/reward_model/ckpt/ckpt_best.pth.tar
Saved reward model ckpt in bipedalwalker_sac_gail_seed0_240229_005622/reward_model/ckpt/ckpt_last.pth.tar


DI-engine DRL Policy
ContinuousQAC(
  (actor_encoder): Identity()
  (critic_encoder): Identity()
  (actor_head): Sequential(
    (0): Linear(in_features=24, out_features=128, bias=True)
    (1): ReLU()
    (2): ReparameterizationHead(
      (main): Sequential(
        (0): Linear(in_features=128, out_features=128, bias=True)
        (1): ReLU()
      )
      (mu): Linear(in_features=128, out_features=4, bias=True)
      (log_sigma_layer): Linear(in_features=128, out_features=4, bias=True)
    )
  )
  (critic_head): ModuleList(
    (0-1): 2 x Sequential(
      (0): Linear(in_features=28, out_features=128, bias=True)
      (1): ReLU()
      (2): RegressionHead(
        (main): Sequential(
          (0): Linear(in_features=128, out_features=128, bias=True)
          (1): ReLU()
        )
        (last): Linear(in_features=128, out_features=1, bias=True)
      )
    )
  )
  (actor): ModuleList(
    (0): Identity()
    (1): Sequential(
      (0): Linear(in_features=24, out_features=128, bia

In [80]:
!zip -r bipedalwalker_gail_sac_0229.zip bipedalwalker_sac_gail_seed0_240229_005622

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


  adding: bipedalwalker_sac_gail_seed0_240229_005622/ (stored 0%)
  adding: bipedalwalker_sac_gail_seed0_240229_005622/total_config.py (deflated 72%)
  adding: bipedalwalker_sac_gail_seed0_240229_005622/log/ (stored 0%)
  adding: bipedalwalker_sac_gail_seed0_240229_005622/log/collector/ (stored 0%)
  adding: bipedalwalker_sac_gail_seed0_240229_005622/log/collector/collector_logger.txt (deflated 79%)
  adding: bipedalwalker_sac_gail_seed0_240229_005622/log/buffer/ (stored 0%)
  adding: bipedalwalker_sac_gail_seed0_240229_005622/log/buffer/buffer_logger.txt (deflated 97%)
  adding: bipedalwalker_sac_gail_seed0_240229_005622/log/buffer/buffer_tb_logger/ (stored 0%)
  adding: bipedalwalker_sac_gail_seed0_240229_005622/log/buffer/buffer_tb_logger/events.out.tfevents.1709172398.61bfd91e8b4a (deflated 75%)
  adding: bipedalwalker_sac_gail_seed0_240229_005622/log/buffer/buffer_tb_logger/events.out.tfevents.1709168367.61bfd91e8b4a (deflated 78%)
  adding: bipedalwalker_sac_gail_seed0_240229_005

In [ ]:
# 需要练一个SAC的bipedalwalk模型出来！

In [75]:
# 这是ppo的
from easydict import EasyDict

bipedalwalker_ppo_config = dict(
    exp_name='bipedalwalker_ppo_seed0',
    env=dict(
        env_id='BipedalWalker-v3',
        collector_env_num=8,
        evaluator_env_num=5,
        # (bool) Scale output action into legal range.
        act_scale=True,
        n_evaluator_episode=5,
        stop_value=300,
        rew_clip=True,
        # The path to save the game replay
        # replay_path='./bipedalwalker_ppo_seed0/video',
    ),
    policy=dict(
        cuda=False,
        load_path="./bipedalwalker_ppo_seed0/ckpt/ckpt_best.pth.tar",
        action_space='continuous',
        model=dict(
            action_space='continuous',
            obs_shape=24,
            action_shape=4,
        ),
        learn=dict(
            epoch_per_collect=10,
            batch_size=64,
            learning_rate=0.001,
            value_weight=0.5,
            entropy_weight=0.01,
            clip_ratio=0.2,
            adv_norm=True,
        ),
        collect=dict(
            n_sample=2048,
            unroll_len=1,
            discount_factor=0.99,
            gae_lambda=0.95,
        ),
    ),
)
bipedalwalker_ppo_config = EasyDict(bipedalwalker_ppo_config)
#main_config = bipedalwalker_ppo_config
bipedalwalker_ppo_create_config = dict(
    env=dict(
        type='bipedalwalker',
        import_names=['dizoo.box2d.bipedalwalker.envs.bipedalwalker_env'],
    ),
    env_manager=dict(type='subprocess'),
    policy=dict(type='ppo'),
)
bipedalwalker_ppo_create_config = EasyDict(bipedalwalker_ppo_create_config)
#create_config = bipedalwalker_ppo_create_config

In [70]:
#from dizoo.box2d.bipedalwalker.config import bipedalwalker_ppo_config , bipedalwalker_ppo_create_config
from ding.config import compile_config
cfg = compile_config(bipedalwalker_ppo_config, create_cfg=bipedalwalker_ppo_create_config, auto=True)

In [71]:
from ding.envs import DingEnvWrapper, BaseEnvManagerV2

collector_env = BaseEnvManagerV2(
    env_fn=[lambda: DingEnvWrapper(gym.make("BipedalWalker-v3")) for _ in range(cfg.env.collector_env_num)],
    cfg=cfg.env.manager
)
evaluator_env = BaseEnvManagerV2(
    env_fn=[lambda: DingEnvWrapper(gym.make("BipedalWalker-v3")) for _ in range(cfg.env.evaluator_env_num)],
    cfg=cfg.env.manager
)

In [73]:
# from ding.model import DQN
from ding.policy import PPOPolicy
from ding.data import DequeBuffer

#model = SAC(**cfg.policy.model)
buffer_ = DequeBuffer(size=cfg.policy.other.replay_buffer.replay_buffer_size)
policy = PPOPolicy(cfg.policy)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [65]:

!mkdir /content/bipedalwalker_sac_config0/ckpt


In [74]:
from ding.framework import task
from ding.framework.context import OnlineRLContext
from ding.framework.middleware import OffPolicyLearner, StepCollector, interaction_evaluator, data_pusher, eps_greedy_handler, CkptSaver

import logging
logging.getLogger().setLevel(logging.INFO)

with task.start(async_mode=False, ctx=OnlineRLContext()):
    # Evaluating, we place it on the first place to get the score of the random model as a benchmark value
    task.use(interaction_evaluator(cfg, policy.eval_mode, evaluator_env))
    #task.use(eps_greedy_handler(cfg))  # Decay probability of explore-exploit
    task.use(StepCollector(cfg, policy.collect_mode, collector_env))  # Collect environmental data
    task.use(data_pusher(cfg, buffer_))  # Push data to buffer
    task.use(OffPolicyLearner(cfg, policy.learn_mode, buffer_))  # Train the model
    task.use(CkptSaver(policy, cfg.exp_name, train_freq=100))  # Save the model
    # In the evaluation process, if the model is found to have exceeded the convergence value, it will end early here
    task.run()

/usr/local/lib/python3.10/dist-packages/gym/core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:268: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed)` instead.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)


AttributeError: 'EasyDict' object has no attribute 'update_per_collect'

# SAC agent

In [ ]:
!git clone https://github.com/opendilab/huggingface_ding.git
!pip3 install -e ./huggingface_ding/

In [50]:
!git clone https://huggingface.co/OpenDILabCommunity/BipedalWalker-v3-SAC

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Cloning into 'BipedalWalker-v3-SAC'...
remote: Enumerating objects: 28, done.
remote: Total 28 (delta 0), reused 0 (delta 0), pack-reused 28
Unpacking objects: 100% (28/28), 267.23 KiB | 4.69 MiB/s, done.


In [52]:
from ding.bonus import SACAgent
from ding.config import Config
from easydict import EasyDict
import torch

# Pull model from Hugggingface hub
policy_state_dict = torch.load("/content/BipedalWalker-v3-SAC/pytorch_model.bin", map_location=torch.device("cpu"))
cfg = EasyDict(Config.file_to_dict("/content/BipedalWalker-v3-SAC/policy_config.py").cfg_dict)

# Instantiate the agent
agent = SACAgent(
    env_id="BipedalWalker-v3",
    exp_name="BipedalWalker-v3-SAC",
    cfg=cfg.exp_config,
    policy_state_dict=policy_state_dict
)
# Continue training
agent.train(step=5000)
# Render the new agent performance
#agent.deploy(enable_save_replay=True)


/usr/local/lib/python3.10/dist-packages/gym/core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


adv_max,▁▄▆▅█
adv_mean,▃▁█▇▇
approx_kl,▆▅█▁▂
clipfrac,█▅█▁▃
cur_lr,▁▁▁▁▁
entropy_loss,▁▂▅██
env step,▁
episode return mean,▁
episode return std,▁
policy_loss,█▅▇▁▃
train iter,▁


/usr/local/lib/python3.10/dist-packages/gym/core.py:268: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed)` instead.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:268: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed)` instead.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:268: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed)` instead.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:268: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed)` instead.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:268: DeprecationWarning: WARN: Function `env

TrainingReturn(wandb_url='https://wandb.ai/anony-mouse-586513020729122229/BipedalWalker-v3-SAC_240228_202927?apiKey=6b60cd7681f9349f2cc972ab9211dc5aab0e7bfd')